In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader,Dataset
from torch import nn
from tqdm import tqdm

In [2]:
df = pd.read_csv('C:\\Users\\qml\\Desktop\\学习\\reinforcement learning for Fintech\\Ubiquant Market Prediction\\train.csv')


In [3]:
y = df["target"]
id=df["investment_id"]
feature = df.drop(['row_id', 'time_id', 'target',"investment_id"], axis=1)


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
feature=torch.from_numpy(feature.values)
id=torch.from_numpy(id.values)
y=torch.from_numpy(y.values)
class FinData(Dataset):
    def __init__(self,feature,id,y):
        self.feature = feature.squeeze().to(device)
        self.y = y.squeeze().to(device)
        self.id = id.squeeze().to(torch.int64).to(device)
        self.len = len(y)

    def __getitem__(self,idx):
        return self.id[idx],self.feature[idx],self.y[idx]
  
    def __len__(self):
        return self.len
Dataset_train=FinData(feature=feature,id=id,y=y)

DataLoader=DataLoader(Dataset_train,batch_size=4096,shuffle=True)

In [5]:
from torch import nn
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.embedding=nn.Embedding(4000,32)
        self.linear1=nn.Linear(32,64)
        self.linear2=nn.Linear(64,64)
        self.linear3=nn.Linear(300,256)
        self.linear4=nn.Linear(256,256)
        self.linear5=nn.Linear(320,512)
        self.linear6=nn.Linear(512,128)
        self.linear7=nn.Linear(128,32)
        self.linear8=nn.Linear(32,1)
        self.linear9=nn.Linear(32,2)
        self.dropout=nn.Dropout(p=0.5)
        self.linear10=nn.Linear(32,64)
        self.linear11=nn.Linear(320,64)
        self.linear12=nn.Linear(64,2)
        
      

    def forward(self, feature, id):
        id1=self.embedding(id).squeeze()
        id1=self.linear1(id1).squeeze()
        id1=id1*torch.sigmoid(id1)
        id1=self.linear2(id1).squeeze()
        id1=id1*torch.sigmoid(id1)
        id1=self.dropout(id1)
        id1=self.linear2(id1).squeeze()
        id1=id1*torch.sigmoid(id1)


        feature1=self.linear3(feature).squeeze()
        feature1=feature1*torch.sigmoid(feature1)
        feature1=self.linear4(feature1).squeeze()
        feature1=feature1*torch.sigmoid(feature1)
        feature1=self.linear4(feature1).squeeze()
        feature1=feature1*torch.sigmoid(feature1)

        x=torch.cat((id1,feature1),1).to("cuda")
        x1=self.linear5(x).squeeze()
        x1=x1*torch.sigmoid(x1)
        x1=self.linear6(x1).squeeze()
        x1=x1*torch.sigmoid(x1)

        x1=self.linear7(x1).squeeze()
        x1=x1*torch.sigmoid(x1)

        x1=self.linear8(x1).squeeze()
        x1=20*torch.sigmoid(x1)


        id2=self.embedding(id).squeeze()
        id2=self.linear10(id2).squeeze()
        id2=self.dropout(id2)

        feature2=self.linear3(feature).squeeze()
        feature2=self.dropout(feature2)


        x2=self.linear11(x).squeeze()
        x2=self.linear12(x2)
        x2=torch.sigmoid(x2)

 








 
        return x1,x2
        #here is x1 is the regression and x2 is the +/- sign


In [6]:
model = NeuralNetwork().to(device)
model=model.double()

In [7]:
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5)
loss_fn1=torch.nn.MSELoss()
loss_fn2=torch.nn.CrossEntropyLoss()

In [8]:
def train_1(dataloader, model, optimizer):
    for  id,feature,y in tqdm(dataloader):
        # Compute prediction and loss
        x1,label = model(feature,id)
        x1=x1.to("cuda")
        label=label.to("cuda")
        #loss1 = loss_fn1(x1,torch.abs(y))
        loss1 = -torch.corrcoef(torch.stack((x1,torch.abs(y))))[0,1]
        loss2 = loss_fn2(label,(0.5*(y/torch.abs(y)+torch.tensor([1]).to(device))).long().to("cuda"))
        loss = loss1+0.1*loss2
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [9]:
def test(dataloader, model):
    scores=[]
    for id, feature,y in tqdm(dataloader):
        cat=[]
        with torch.no_grad():
            x1,x2=model(feature,id)
            for cats in x2:
                if cats[0]>cats[1]:
                    cat.append(-1)
                else:
                    cat.append(1)
        cat=torch.tensor(cat).to("cuda")
        result=cat*x1
        score=torch.corrcoef(torch.stack((result,y)))[0,1].item()
        scores.append(score)
    score_tensor=torch.tensor(scores)
    mean=torch.mean(score_tensor)
    return mean
            



In [10]:
for i in range(5):
    print("epoch{}/{}".format(i+1,10))
    train(DataLoader,model,optimizer)


epoch1/10


100%|██████████| 767/767 [01:20<00:00,  9.51it/s]


epoch2/10


100%|██████████| 767/767 [01:19<00:00,  9.68it/s]


epoch3/10


100%|██████████| 767/767 [01:18<00:00,  9.73it/s]


epoch4/10


100%|██████████| 767/767 [01:19<00:00,  9.68it/s]


epoch5/10


100%|██████████| 767/767 [01:19<00:00,  9.70it/s]


In [11]:
test(DataLoader,model)

100%|██████████| 767/767 [03:14<00:00,  3.95it/s]


tensor(0.0835)

In [12]:
torch.save(model,"all feature with auxliary.pth")